In [1]:
import re
import pandas as pd
import numpy as np
import MySQLdb
import keras.models
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import shift
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold, KFold
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Dropout, LSTM, Merge, Input, Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.utils import np_utils
from sklearn.externals import joblib
from sqlalchemy import create_engine

Using TensorFlow backend.


In [2]:
def binary(Y):
    Y[np.where(Y > 0)] = 1
    Y[np.where(Y <= 0)] = 0
    Y = Y.astype('int64')
    f = np.bincount(Y)
    print(f/np.sum(f))
    return Y

def get_shift(data, shift_offset=2):
    g = data.as_matrix(columns=['gbp_gradient']).reshape(-1)
    for i in range(1, shift_offset+1):
        data['gbp_gradient_p_'+str(i)] = shift(g, i, cval=3)

In [3]:
tfidfvectorizer = joblib.load('tfidf_vectorizer.pkl')
model = keras.models.load_model('model.h5')

In [4]:
data = pd.read_csv('newstitle_gbp(uk-news,world,business,technology,money_2017-5-31).csv')
data['gbp_gradient'] = np.gradient(data.as_matrix(columns=['gbp']).reshape(-1))
get_shift(data, shift_offset=2)
data.head()

,timestamp,title,gbp,gbp_gradient,gbp_gradient_p_1,gbp_gradient_p_2
0,1483286400,Briton killed while fighting against Isis in S...,1.229379,-0.003644,3.000000,3.000000
1,1483372800,Fiji earthquake tsunami threat passes after 7....,1.225735,0.000282,-0.003644,3.000000
2,1483459200,Marine Le Pen cheers Ford move out of Mexico a...,1.229943,0.007793,0.000282,-0.003644
3,1483545600,Southern rail: RMT agrees to direct talks with...,1.241321,0.000530,0.007793,0.000282
4,1483632000,"Southern rail, London Underground and BA staff...",1.231003,-0.006310,0.000530,0.007793


In [5]:
X = tfidfvectorizer.transform(data['title'].tolist())
X

<151x2730 sparse matrix of type '<class 'numpy.float64'>'
	with 9250 stored elements in Compressed Sparse Row format>

In [6]:
Y = data.as_matrix(columns=['gbp_gradient']).reshape(-1)
#Y = shift(Y, -1, cval=0)
num_classes = 2
Y = binary(Y)
y = np_utils.to_categorical(Y, num_classes)

[ 0.45695364  0.54304636]


In [7]:
# output = pd.DataFrame()
# output['timestamp'] = data['timestamp']
# output['gbp'] = data['gbp']
# output['gbp_gradient'] = data['gbp_gradient']
# output['gbp_gradient_binary'] = Y
# binary_pred = np.zeros(151)

In [8]:
X_p = data.as_matrix(columns=['gbp_gradient_p_1', 'gbp_gradient_p_2'])
X_p[:,0] = binary(X_p[:,0])
X_p[:,1] = binary(X_p[:,1])
X = hstack([X, X.power(2), X.power(3), X.power(4), X_p])
X = X.toarray()
X.shape

[ 0.45033113  0.54966887]
[ 0.45033113  0.54966887]


(151, 10922)

In [9]:
score = model.evaluate(X, y, batch_size=500)
score

151/151 [==============================] - 0s


[0.6722874641418457, 0.61589407920837402]

In [10]:
pred = np.argmax(model.predict(X), axis=1)
target_names = ['decline','up']
print(classification_report(Y, pred, target_names=target_names))

             precision    recall  f1-score   support

    decline       0.60      0.48      0.53        69
         up       0.62      0.73      0.67        82

avg / total       0.61      0.62      0.61       151



In [11]:
# output['gbp_gradient_binary_pred'] = np.argmax(model.predict(X), axis=1)
# output.to_pickle('output_2017.pkl')